In [1]:
import tensorflow

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras

Using TensorFlow backend.


In [3]:
print(1)

1


In [4]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [5]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7438121156547839861
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14367002935416756319
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [8]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [51]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [33]:
nr_categories = 102
batch_size = 16
epochs = 50

In [34]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_fungi_all_p2.h5")

In [35]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_2
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_conv4
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_conv4
16 block4_pool
17 block5_conv1
18 block5_conv2
19 block5_conv3
20 block5_conv4
21 block5_pool


In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [37]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model.layers[:]:
    print(layer.name)
    layer.trainable = False

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool


In [38]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [39]:
model_final = Model(input = model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [40]:
model_final.shape

AttributeError: 'Model' object has no attribute 'shape'

In [ ]:
# Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model_final.layers[:12]:
    print(layer.name)
    layer.trainable = False

In [41]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [42]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [47]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2)

In [48]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [56]:
img_width, img_height = 256, 256
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 4604 images belonging to 102 classes.


In [57]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 1094 images belonging to 102 classes.


In [58]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_oxford102_train_dataset.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [59]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [60]:
#batch 16

In [61]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
288/287 [==============================] - 88s 307ms/step - loss: 4.5757 - acc: 0.0380 - val_loss: 4.3221 - val_acc: 0.0521

Epoch 00001: val_acc improved from -inf to 0.05210, saving model to vgg16_oxford102_train_dataset.h5
Epoch 2/50
288/287 [==============================] - 83s 289ms/step - loss: 4.2929 - acc: 0.0735 - val_loss: 4.0831 - val_acc: 0.1243

Epoch 00002: val_acc improved from 0.05210 to 0.12431, saving model to vgg16_oxford102_train_dataset.h5
Epoch 3/50
288/287 [==============================] - 83s 289ms/step - loss: 4.0673 - acc: 0.1171 - val_loss: 3.9103 - val_acc: 0.1600

Epoch 00003: val_acc improved from 0.12431 to 0.15996, saving model to vgg16_oxford102_train_dataset.h5
Epoch 4/50
288/287 [==============================] - 84s 291ms/step - loss: 3.8164 - acc: 0.1643 - val_loss: 3.6347 - val_acc: 0.2011

Epoch 00004: val_acc improved from 0.15996 to 0.20110, saving model to vgg16_oxford102_train_dataset.h5
Epoch 5/50
288/287 [=======================


Epoch 00037: val_acc improved from 0.72121 to 0.72761, saving model to vgg16_oxford102_train_dataset.h5
Epoch 38/50
288/287 [==============================] - 84s 292ms/step - loss: 0.7040 - acc: 0.8620 - val_loss: 1.2300 - val_acc: 0.7230

Epoch 00038: val_acc did not improve from 0.72761
Epoch 39/50
288/287 [==============================] - 83s 289ms/step - loss: 0.6504 - acc: 0.8749 - val_loss: 1.2266 - val_acc: 0.7194

Epoch 00039: val_acc did not improve from 0.72761
Epoch 40/50
288/287 [==============================] - 83s 289ms/step - loss: 0.6474 - acc: 0.8723 - val_loss: 1.2208 - val_acc: 0.7203

Epoch 00040: val_acc did not improve from 0.72761
Epoch 41/50
288/287 [==============================] - 83s 288ms/step - loss: 0.6152 - acc: 0.8831 - val_loss: 1.1743 - val_acc: 0.7331

Epoch 00041: val_acc improved from 0.72761 to 0.73309, saving model to vgg16_oxford102_train_dataset.h5
Epoch 42/50
288/287 [==============================] - 83s 290ms/step - loss: 0.5914 - acc: 0

In [ ]:
#batch 256


In [ ]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

In [ ]:
!ls

In [ ]:
batch_size = 16

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/10),
        class_mode=None) #,  # this means our generator will only yield batches of data, no labels
#) #shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, 60000)
# save the output as a Numpy array
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/20),
        class_mode=None)
#) #shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, 8000)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open('bottleneck_features_train.npy'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nr_categories, activation='softmax')) 
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

In [ ]:
print(1)

In [19]:
indices_inv = train_generator.class_indices

In [20]:
indices = {v:k for k, v in indices_inv.items()}

In [27]:
indices[36]

'10249_Amanita_citrina var. citrina'

In [24]:
indices_inv['10273_Amanita_virosa']

45

In [7]:
from PIL import Image

im = Image.open('fgcvx/train/images/15309_Inocybe_adaequata/TOB2017-9198446_HypgMYRSZ.JPG')
width, height = im.size
print(width, height)


941 800


1000 667


In [3]:
os.getcwd()

'/data'